In [ ]:
import os
import torchaudio
import pandas as pd
import matplotlib.pyplot as plt

# 데이터셋 디렉토리 지정
data_dir = "/data/seungmin/dataset/k-12_training_processed/number_or_english"

# WAV 파일 길이를 저장할 리스트
wav_lengths = []
missing_files = []

# 파일 리스트 가져오기
for root, _, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".wav"):
            wav_path = os.path.join(root, file)
            try:
                # WAV 파일 로드 및 길이 계산
                waveform, sample_rate = torchaudio.load(wav_path)
                duration = waveform.shape[1] / sample_rate  # 초 단위 변환
                wav_lengths.append(duration)
            except Exception as e:
                print(f"⚠️ {wav_path} 처리 중 오류 발생: {e}")
                missing_files.append(wav_path)

# 데이터프레임 생성
df = pd.DataFrame({"speech_length": wav_lengths})

# 기본 통계 출력
print("📌 총 매칭된 wav 파일 개수:", len(wav_lengths))
print(df["speech_length"].describe(percentiles=[0.95, 0.99]))

# 히스토그램 시각화
plt.figure(figsize=(10, 5))
plt.hist(df["speech_length"], bins=50, alpha=0.75, edgecolor="black")
plt.xlabel("wav length (sec)")
plt.ylabel("file nums")
plt.title("k-12 wav file length distribution")
plt.grid(True)
plt.show()

# 누락된 wav 파일 개수 확인
print(f"❌ 존재하지 않는 wav 파일 개수: {len(missing_files)}")
if missing_files:
    print("❌ 누락된 일부 파일 샘플:", missing_files[:5])
